<a href="https://colab.research.google.com/github/apple-ic/Agentic/blob/main/Content_writer_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Overall Task for Sudhir**

***Content Writer***

***Role*** : Copywriter

***Backstory*** : Professional copywriter with experience in long-form and short-form content. Strong grammar and storytelling skills.

***Goal*** : Write clear, engaging content based on selected campaign ideas (e.g., blog posts, ad copy, social captions).

## **Individual agents**

*Note to Team: This task is split into individual agents as according to a reddit post, if an agent has more than 2-3 individual tasks, it begins to hallucinate*

- Labelling agent for different campaigns
- Long-form agent
- Short-form agent

*Note to Team: A medium Article splits long form content to be equal or more than 1,500 words and short-form content to be less than 1,000 words, which is why there are separate agents for short form and long form content*

## **Agents, Backstories and Goals**

### Labelling Agent

*Note to Self/Team: Labelling Agent operates on the assumption that the creative director's output is in JSON format (edits may have to be made if the output is Pydantic)*

*Updated Note to Self: This agent and it's associated tasks may be unnecessary if the creative director's output includes whether the platform is geared towards short or long form content*

***Role*** : Labelling creative director's output

***Backstory*** : You are experienced to tell the difference which marketing platform needs short form or long form content

***Goal*** : Parse through the creative director's output. Generate two JSON files: short-form, long-form. Short-form file contains the campaigns that are for short form platforms and long-form file contains the campaigns that are for long form platforms


### Short Form Agent

***Role*** : Copywriter for Short Form content

***Backstory*** : Professional copywriter with experience in short-form content. Strong grammar and storytelling skills.

***Goal*** : Write clear and engaging content on {{topic}} that is optimised for {{platform}}. Keep the generated content less than 1,000 words. Generate relevant images alongside the copy.

### Long Form Agent

***Role*** : Copywriter for Long Form content

***Backstory***: Professional copywriter with experience in long-form content. Strong grammar and storytelling skills

***Goal*** : Write clear and engaging content on {{topic}} that is optimised for {{platform}}. Keep the generated content equal or more than 1,500 words. Generate relevant images alongside the copy.

## **Tasks**

### Tasks for Labelling Agent

#####Task 1
***Assigned to*** : Labelling Agent

***Input*** : Creative Director's JSON file

***Output*** : Updated JSON file with an extra line added denoting whether the platform is short or long form

#####Task 2

***Assigned to*** : Labelling Agent

***Input*** : Updated JSON file

***Output***: Two JSON files, short-form and long-form


### Tasks for Short Form Agent

#####Task 1

***Assigned to*** : Short Form Agent

***Input***: Short Form JSON File

***Output***: Document with relevant content

#####Task 2

***Assigned to*** : Short Form Agent

***Input***: Document with previous content and editor agent's comments

***Output***: Revised Document


### Tasks for Long Form Agent

#####Task 1

***Assigned to*** : Long Form Agent

***Input***: Long Form JSON File

***Output***: Document with relevant content

#####Task 2

***Assigned to*** : Long Form Agent

***Input***: Document with previous content and editor agent's comments

***Output***: Revised Document

In [ ]:
%pip install crewai

In [ ]:
#Declaring copywriting Agents

from crewai import Agent

labeling_agent = Agent(
    role = "Labelling creative director's output",
    goal = "Parse through the creative director's output. Generate two JSON files: short-form, long-form. Short-form file contains the campaigns that are for short form platforms and long-form file contains the campaigns that are for long form platforms",
    backstory = "You are experienced to tell the difference which marketing platform needs short form or long form content",
    verbose=True
)


short_form_agent = Agent(
    role = "Copywriter for Short Form content",
    goal = "Write clear and engaging content that is optimised for platforms that use short form content. Keep the generated content less than 1,000 words. Generate relevant images alongside the copy.",
    backstory = "Professional copywriter with experience in short-form content. Strong grammar and storytelling skills.",
    verbose=True
)

long_form_agent = Agent(
    role = "Copywriter for Long Form content",
    goal = "Write clear and engaging content that is optimised for platforms that use long form content. Keep the generated content equal or more than 1,500 words. Generate relevant images alongside the copy.",
    backstory = "Professional copywriter with experience in long-form content. Strong grammar and storytelling skills.",
    verbose=True
)


In [ ]:
#creation of a tool to create multiple files at once
#output_file only writes out one file, need it for the second labelling agent task

from crewai.tools import BaseTool
from pydantic import BaseModel, Field

class MyToolInput(BaseModel):
  """Input Schema for MultipleFileWriterTool"""
  argument: str = Field(..., description="Description of the argument.")

class MultipleFileWriterTool(BaseTool):
  name: str = "MultipleFileWriter"
  description: str="Writes Multiple Files to the same path folder specified at once"
  args_schema: Type[BaseModel] = MyToolInput

  def _run(self, content: str, filepath: str):
    with open(filepath, 'w') as f:
      f.write(content)
    return f"Saved to {filepath}"

In [ ]:
#Declaring Tasks for Copywriting Agents
from crewai import Task

from datetime import datetime

file_path = "" #this variable contains the file path where the files will be stored

current_date = datetime.now().strftime('%Y-%m=%d')
current_time = datetime.now().strftime('%H:%M:%S')

#labelling agents tasks (may need to be scraped)

multiple_file_tool = MultipleFileWriterTool()

adding_type_platform_task = Task(
    agent = labelling_agent,
    description = """
    You are provided with a JSON file containing campaign ideas curated by the Creative Director.
    Your task is to analyze each campaign entry and identify the intended platform's format.
    Based on this, append a new field to each entry that specifies whether it is for a **short-form** or **long-form** platform.

    Be thorough and consistent in your classification. Use logical reasoning to determine the format type.
    """,
    expected_output="An updated JSON file with an extra line added which denotes whether the platform is short or long form",
    output_file= f"{file_path}/campaign_context_updated_{current_date}_{current_time}.json"
)

creation_of_platform_type_files_task = Task(
    agent = labelling_agent,
    description = """
     Take the updated campaign JSON file, which includes platform format labels, and split it into two separate JSON files:

    - One file should contain **only short-form** campaign entries.
    - The other should contain **only long-form** campaign entries.

    Ensure that both output files are properly formatted and include all relevant fields from the original entries.
    """,
    expected_output = "Two JSON files are outputed, one containing the campaign ideas for short form platforms and the other contraining the campaign ideas for long form platforms"
    tools = [multiple_file_tool],
    allow_delegation = False
)

#short form agents tasks

short_form_content_task = Task(
    agent = short_form_agent,
    description = """
    Using the provided short-form JSON file, generate engaging and concise content for each campaign entry.

    Focus on platforms such as TikTok, Instagram Reels, YouTube Shorts, etc. Your writing should be punchy, attention-grabbing, and tailored for fast-scrolling audiences.

    Incorporate campaign themes, tone, and any specified style guidelines.
    """,
    expected_output=f"{file_path}/short_form_content_{current_date}_{current_time}.txt"
)

short_form_revised_content_task = Task(
    agent = short_form_agent,
    description = """
    You will receive a document containing your previously written short-form content along with feedback from the Editor Agent.

    Review the comments and revise your content accordingly. Ensure all suggestions are addressed while keeping the content concise and aligned with platform norms.

    Your final output should be cleaner, more impactful, and free of errors.
    """,
    expected_output=f"{file_path}/short_form_content_revised_{current_date}_{current_time}.txt"
)

#long form agents tasks

long_form_content_task = Task(
    agent = long_form_agent,
    description = """
    Use the long-form JSON file to generate detailed and structured content pieces.

    These might be blog posts, articles, or long-form social media posts for platforms like LinkedIn or Medium.
    Be informative, persuasive, and aligned with the campaign's voice. Use storytelling or thought leadership approaches when relevant.
    """,
    expected_output=f"{file_path}/long_form_content_{current_date}_{current_time}.txt"
)

long_form_revised_content_task = Task(
    agent = long_form_agent,
    description = """
    Revise your initial long-form content using the Editor Agent's comments and suggestions.

    Focus on enhancing clarity, structure, and tone while preserving the message and style.
    Ensure the final content is polished, cohesive, and ready for publication.
    """,
    expected_output=f"{file_path}/long_form_content_{current_date}_{current_time}.txt"
)

In [ ]:
#Combining the Agents together into one pipeline (could use this as the basis for the rest of the pipeline)